In [ ]:
# DONT RUNT THIS CELL
# moved to the beginning to not lost it in case we need it later


#drop columns i dont care or have too many empty values:
aux_training = aux_training.drop('date_time', axis=1)

aux_training = aux_training.drop('comp1_rate', axis=1)
aux_training = aux_training.drop('comp1_inv', axis=1)
aux_training = aux_training.drop('comp1_rate_percent_diff', axis=1)

aux_training = aux_training.drop('comp2_rate', axis=1)
aux_training = aux_training.drop('comp2_inv', axis=1)
aux_training = aux_training.drop('comp2_rate_percent_diff', axis=1)

aux_training = aux_training.drop('comp3_rate', axis=1)
aux_training = aux_training.drop('comp3_inv', axis=1)
aux_training = aux_training.drop('comp3_rate_percent_diff', axis=1)

aux_training = aux_training.drop('comp4_rate', axis=1)
aux_training = aux_training.drop('comp4_inv', axis=1)
aux_training = aux_training.drop('comp4_rate_percent_diff', axis=1)

aux_training = aux_training.drop('comp5_rate', axis=1)
aux_training = aux_training.drop('comp5_inv', axis=1)
aux_training = aux_training.drop('comp5_rate_percent_diff', axis=1)

aux_training = aux_training.drop('comp6_rate', axis=1)
aux_training = aux_training.drop('comp6_inv', axis=1)
aux_training = aux_training.drop('comp6_rate_percent_diff', axis=1)

aux_training = aux_training.drop('comp7_rate', axis=1)
aux_training = aux_training.drop('comp7_inv', axis=1)
aux_training = aux_training.drop('comp7_rate_percent_diff', axis=1)

aux_training = aux_training.drop('comp8_rate', axis=1)
aux_training = aux_training.drop('comp8_inv', axis=1)
aux_training = aux_training.drop('comp8_rate_percent_diff', axis=1)

aux_training = aux_training.drop('visitor_hist_starrating', axis=1)
aux_training = aux_training.drop('visitor_hist_adr_usd', axis=1)

aux_training = aux_training.drop('srch_query_affinity_score', axis=1)

aux_training = aux_training.drop('gross_bookings_usd', axis=1)

aux_training = aux_training.drop('prop_review_score', axis=1)
aux_training = aux_training.drop('prop_location_score2', axis=1)
aux_training = aux_training.drop('orig_destination_distance', axis=1)

In [ ]:
pip install lightgbm

In [1]:
import lightgbm as lgb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy
import gc
from sklearn.preprocessing import StandardScaler,OneHotEncoder,LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import ndcg_score

In [ ]:
test_data  = pd.read_csv("test_set_VU_DM.csv")

In [2]:
training_data  = pd.read_csv("training_set_VU_DM.csv")

In [3]:
#create new column for NDCG evaluation
training_data['target_col'] = training_data.apply(
    lambda row: (5 if row.booking_bool == 1 else (1 if row.click_bool == 1 else 0)), axis=1)

In [ ]:
#del test_data
#del aux_training
#gc.collect()

In [ ]:
#aux_training = copy.deepcopy(training_data)
#aux_training.head()

In [4]:
aux_training = copy.copy(training_data) #shallow copy for now because of memory problems
aux_training.head()

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,click_bool,gross_bookings_usd,booking_bool,target_col
0,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,893,3,3.5,...,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0,0
1,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,10404,4,4.0,...,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0,0
2,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,21315,3,4.5,...,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0,0
3,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,27348,2,4.0,...,NaN,NaN,NaN,-1.0,0.0,5.0,0,NaN,0,0
4,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,29604,4,3.5,...,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0,0


In [5]:
aux_training = aux_training.drop('date_time', axis=1) # cause me dont care datee

In [6]:
##  drop columns that are not in test set (??? ) 
aux_training = aux_training.drop('click_bool', axis=1)   #replaced by target_col and not in test set 
aux_training = aux_training.drop('booking_bool', axis=1) #replaced by target_col and not in test set 

aux_training = aux_training.drop('position', axis=1)     # not in test set 
aux_training = aux_training.drop('gross_bookings_usd', axis=1) # not in test set


In [ ]:
print("Empty values per column: \n",aux_training.isnull().sum())

In [7]:
feature = aux_training.drop('target_col', axis=1)
target  =  aux_training['target_col']

In [31]:
X_train, X_test, y_train, y_test = train_test_split(feature, target, test_size=0.2, random_state=1)

print("X_train.shape = ",X_train.shape)
print("X_test.shape = ",X_test.shape)
print("\ny_train.shape = ",y_train.shape)
print("y_test.shape = ",y_test.shape)

X_train.shape =  (3966677, 49)
X_test.shape =  (991670, 49)

y_train.shape =  (3966677,)
y_test.shape =  (991670,)


In [32]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)

print("X_train.shape = ",X_train.shape)
print("X_val.shape = ",X_val.shape)
print("\ny_train.shape = ",y_train.shape)
print("y_val.shape = ",y_val.shape)

X_train.shape =  (3173341, 49)
X_val.shape =  (793336, 49)

y_train.shape =  (3173341,)
y_val.shape =  (793336,)


In [33]:
X_train = X_train.sort_index()
y_train = y_train.sort_index()

X_val = X_val.sort_index()
y_val = y_val.sort_index()

X_test = X_test.sort_index()
y_test = y_test.sort_index()

In [57]:
# get number of rows of each query (srch_id). Needed to train the model.
qids_train = X_train.groupby("srch_id")["srch_id"].count().to_numpy() 
qids_val   = X_val.groupby("srch_id")["srch_id"].count().to_numpy()
qids_test   = X_test.groupby("srch_id")["srch_id"].count().to_numpy()

qids_test


array([9, 9, 1, ..., 3, 6, 1], dtype=int64)

In [ ]:
print("query train ids mean: ",qids_train.mean()) 
print("query val   ids mean: ",qids_val.mean()) 

In [35]:
y_test #---->> THIS ARE THE TRUE VALUES WE NEED TO COMPARE AFTER MAKING PREDICTION OVER X_TEST

5          0
7          0
9          0
12         5
15         0
          ..
4958326    0
4958328    0
4958332    0
4958339    0
4958342    0
Name: target_col, Length: 991670, dtype: int64

In [52]:
FIXED_PARAMS={
              #'objective'             : 'lambdarank',
              'objective'               : 'rank_xendcg',
              'metric'                : 'ndcg',
              'boosting_type'         : 'gbdt',
              'is_unbalance'          : False,
              'num_iterations'        : 500,
              'early_stopping_rounds' : 50}

SEARCH_PARAMS = {'learning_rate': 0.15,
                 'max_depth': 15,
                 'num_leaves': 25,
                 'feature_fraction': 0.8,
                 'subsample': 0.2}

gbm = lgb.LGBMRanker( 
        objective               = FIXED_PARAMS["objective"],
        metric                  = FIXED_PARAMS["metric"],
        boosting_type           = FIXED_PARAMS["boosting_type"],
        is_unbalance            = FIXED_PARAMS["is_unbalance"],
        num_iterations          = FIXED_PARAMS["num_iterations"],
        early_stopping_rounds   = FIXED_PARAMS["early_stopping_rounds"],
        learning_rate           = SEARCH_PARAMS["learning_rate"],
        max_depth               = SEARCH_PARAMS["max_depth"],
        num_leaves              = SEARCH_PARAMS["num_leaves"],
        feature_fraction        = SEARCH_PARAMS["feature_fraction"],
        subsample               = SEARCH_PARAMS["subsample"],
)

# gbm = lgb.LGBMRanker( params )



In [53]:
# Training
gbm.fit(X_train, 
        y_train, 
        group=qids_train,
        eval_set=[(X_val, y_val)], 
        eval_group=[qids_val],
        eval_at=[5]
)

[1]	valid_0's ndcg@5: 0.947869
[2]	valid_0's ndcg@5: 0.949048
[3]	valid_0's ndcg@5: 0.949969
[4]	valid_0's ndcg@5: 0.950522
[5]	valid_0's ndcg@5: 0.950879
[6]	valid_0's ndcg@5: 0.951285
[7]	valid_0's ndcg@5: 0.951544
[8]	valid_0's ndcg@5: 0.952133
[9]	valid_0's ndcg@5: 0.952013
[10]	valid_0's ndcg@5: 0.952344
[11]	valid_0's ndcg@5: 0.952617
[12]	valid_0's ndcg@5: 0.95295
[13]	valid_0's ndcg@5: 0.953001
[14]	valid_0's ndcg@5: 0.953167
[15]	valid_0's ndcg@5: 0.953246
[16]	valid_0's ndcg@5: 0.953356
[17]	valid_0's ndcg@5: 0.953433
[18]	valid_0's ndcg@5: 0.95351
[19]	valid_0's ndcg@5: 0.95364
[20]	valid_0's ndcg@5: 0.953748
[21]	valid_0's ndcg@5: 0.953763
[22]	valid_0's ndcg@5: 0.95381
[23]	valid_0's ndcg@5: 0.95387
[24]	valid_0's ndcg@5: 0.954018
[25]	valid_0's ndcg@5: 0.954156
[26]	valid_0's ndcg@5: 0.954179
[27]	valid_0's ndcg@5: 0.954329
[28]	valid_0's ndcg@5: 0.954431
[29]	valid_0's ndcg@5: 0.954461
[30]	valid_0's ndcg@5: 0.954522
[31]	valid_0's ndcg@5: 0.954595
[32]	valid_0's ndcg@5:

LGBMRanker(early_stopping_rounds=50, feature_fraction=0.8, is_unbalance=False,
           learning_rate=0.15, max_depth=15, metric='ndcg', num_iterations=500,
           num_leaves=25, objective='rank_xendcg', subsample=0.2)

------------------------------------------------------------------------------------------------------------
## prediction over crossover set to get an estimate of NDCG evaluation

In [55]:
#X_test = X_test.drop("predicted_ranking",axis= 1)

In [48]:
# Making prediction over crossover set (ranking)
test_pred = gbm.predict(X_test)

print(test_pred)

[ 0.02735438 -1.42372443 -0.85672676 ...  0.85929939  0.12412567
 -0.06921279]


In [58]:
X_test.head(50)

,srch_id,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,...,comp5_rate_percent_diff,comp6_rate,comp6_inv,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff
5,1,12,187,NaN,NaN,219,30184,4,4.5,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,7.0
7,1,12,187,NaN,NaN,219,50984,2,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1,12,187,NaN,NaN,219,56880,4,4.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN
12,1,12,187,NaN,NaN,219,68914,2,3.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,11.0
15,1,12,187,NaN,NaN,219,85728,2,3.0,1,...,14.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN
18,1,12,187,NaN,NaN,219,88218,4,3.5,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN
21,1,12,187,NaN,NaN,219,95307,4,3.5,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,0.0,27.0
24,1,12,187,NaN,NaN,219,111000,3,4.5,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN
27,1,12,187,NaN,NaN,219,122844,3,4.5,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN
30,4,5,219,NaN,NaN,219,11826,5,4.5,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
X_test["predicted_ranking"] = test_pred
#X_test[["srch_id","prop_id","predicted_ranking"]].head(50)

In [21]:
y_test.head(50)

5      0
7      0
9      0
12     5
15     0
18     0
21     0
24     0
27     0
30     0
32     0
33     0
36     0
44     0
45     0
49     0
55     0
57     0
62     0
67     0
79     0
82     0
83     0
112    0
118    0
122    0
129    0
134    0
135    0
143    0
147    0
160    0
165    0
168    1
174    0
181    0
186    0
188    0
207    0
208    0
211    5
217    0
219    0
224    0
228    0
234    0
235    0
240    0
243    5
247    0
Name: target_col, dtype: int64

In [63]:
# This is our own evaluation of the model, before getting real value in kaggle score
print("--> NDCG : ", ndcg_score([y_test],[X_test["predicted_ranking"]]))

--> NDCG :  0.7816386460917436


------------------------------------------------------------------------------------------
## real prediction over test_set and output for kaggle competition

In [ ]:
test_data = test_data.drop('date_time', axis=1)

In [ ]:
# Create prediction over the test set
test_pred = gbm.predict(test_data)

print(test_pred)

In [ ]:
# Add new column "predicted_ranking" to the test_set with predicted values of the model
test_data["predicted_ranking"] = test_pred
test_data.head(15)


In [ ]:
# Sort by srch_id and predicted_ranking
test_data = test_data.sort_values(["srch_id","predicted_ranking"], ascending=[True,False])

In [ ]:
test_data.head()

In [ ]:
# Drop all columns but srch_id and prop_id for output format
test_data.drop(test_data.columns.difference(['srch_id','prop_id']), 1, inplace=True)
test_data.head()

In [ ]:
# Create output file for competition
test_data.to_csv("out2.csv", index=False)    

# -------------------------------------------------------------------------------------------------------------